# Workshop Deep Learning
## Modell Zusammenstellung und Training
### SetUp:

In [1]:
name = 'Jewellery'

In [2]:
# CPU use (before importing tensorflow backend):

#from __future__ import print_function
#import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
# Part 1 - Building the CNN
import pandas
# set Workingdirectory
import os
#os.chdir(r'Path\to\your\pictures')

import random
# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import RMSprop

#from google_images_download import google_images_download   #importing the library
import shutil
import numpy as np

import urllib.request
import time
from selenium import webdriver
import sys


---
# CNN
### Pretrained Convolutional Layers


In [3]:
conv_base_vgg16 = VGG16(
  weights = "imagenet",
  include_top = False,
  input_shape = (150, 150, 3)
)

conv_base_vgg16.trainable = False

conv_base_vgg16.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     


---
### Initialising the CNN

In [4]:

classifier = Sequential()

classifier.add(conv_base_vgg16)

classifier.add(Flatten(input_shape=conv_base_vgg16.output_shape[1:]))

# =============================================================================
# selbst entworfene Convolutional Layers
#
# # Step 1 - Convolution
# classifier.add(Convolution2D(128, kernel_size = (3,3), strides = (3, 3), input_shape = (150,150, 3), activation = 'relu'))
# 
# # Step 2 - Pooling
# classifier.add(MaxPooling2D(pool_size = (2, 2)))
# 
# # Adding a second convolutional layer
# classifier.add(Convolution2D(64, kernel_size = (3,3), strides = (3, 3), activation = 'relu'))
# 
# classifier.add(MaxPooling2D(pool_size = (2, 2)))
# 
# # Step 3 - Flattening
# classifier.add(Flatten())
# 
# =============================================================================
# Step 4 - Full connection

classifier.add(Dense(units = 256, activation = 'relu'))

classifier.add(Dense(units = 64, activation = 'relu'))

classifier.add(Dense(units = 3, activation = 'softmax'))

# Compiling the CNN
#classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.compile(optimizer = RMSprop(lr = 2e-5), loss ='categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 16,828,739
Trainable params: 2,114,051
Non-trainable params: 14,714,688
_________________________________________________________________


---
### Data Generators

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1/255,
                                     rotation_range = 40,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = "nearest")

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(name +'/training_set',
                                                 target_size = [150,150],
                                                 batch_size = 10, 
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(name + '/test_set',
                                            target_size = [150,150],
                                            batch_size = 10, 
                                            class_mode = 'categorical')

classes = training_set.class_indices
f = open(name + '/classes.txt','w')
f.write(str(classes))
f.close()
print(classes)

Found 145 images belonging to 3 classes.
Found 50 images belonging to 3 classes.
{'Bracelet': 0, 'Necklace': 1, 'Ring': 2}


In [6]:
import PIL
i = 0
try:
    os.mkdir(name + '/batch_pics')
except:
    pass

for batch in train_datagen.flow_from_directory(name + '/test_set',
                                      target_size = (150,150),
                                      batch_size = 10,
                                      class_mode = 'categorical',
                                      save_to_dir = name + '/batch_pics'):
    i += 1
    if i > 20: break  

Found 50 images belonging to 3 classes.


---
### checkpoint callback

In [7]:

# Checkpoints des CNN(nach jeder Epoch wird gespeichert):
checkpoint_dir = name + '/checkpoints'
try:
    os.makedirs(checkpoint_dir)
except:
    pass
filepath = checkpoint_dir + "/model.{epoch:02d}-{loss:.3f}-{acc:.2f}-{val_acc:.2f}.hdf5"

# Create checkpoint callback
cp_callback = ModelCheckpoint(
    filepath = filepath,
    save_weights_only = False,
    verbose = 1,
    save_best_only = True
)



---
### TRAIN

In [8]:
###############################################################################
random.seed(1)

classifier.fit_generator(training_set,
                         #steps_per_epoch = 10,
                         epochs = 40,
                         validation_data = test_set,
                         #callbacks = [cp_callback]
                        )



Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 15 steps, validate for 5 steps
Epoch 1/40
15/15 [==============================] - 9s 618ms/step - loss: 1.1846 - accuracy: 0.3862 - val_loss: 1.0262 - val_accuracy: 0.5600
Epoch 2/40
15/15 [==============================] - 8s 543ms/step - loss: 1.0171 - accuracy: 0.5379 - val_loss: 0.8958 - val_accuracy: 0.6200
Epoch 3/40
15/15 [==============================] - 9s 568ms/step - loss: 0.9264 - accuracy: 0.6000 - val_loss: 0.8177 - val_accuracy: 0.7200
Epoch 4/40
15/15 [==============================] - 8s 547ms/step - loss: 0.8322 - accuracy: 0.6759 - val_loss: 0.7635 - val_accuracy: 0.7600
Epoch 5/40
15/15 [==============================] - 9s 571ms/step - loss: 0.7598 - accuracy: 0.7379 - val_loss: 0.7386 - val_accuracy: 0.7000
Epoch 6/40
15/15 [==============================] - 8s 556ms/step - loss: 0.7231 - accuracy: 0.7379 - val_loss: 0.6577 - va

---
### Save Data

In [9]:


classifier.save(name + "/checkpoints/cnn_model.hdf5")
